<a href="https://colab.research.google.com/github/vmartinezarias/Ecoacustica/blob/main/Indices_Acusticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Índices acústicos utilizados, definiciones y fórmulas

Notación base.
La señal de entrada es un audio en mono a frecuencia de muestreo Fs. El espectrograma de magnitud (STFT) se denota como S(f, t), con f = frecuencias y t = ventanas de tiempo. El análisis se restringe a un rango de frecuencias definido por fmin y fmax.

1) ACI — Acoustic Complexity Index

Intuición.
Mide la variación relativa de energía. Señales impulsivas o rápidas (trinos, ráfagas) producen valores altos; señales estacionarias o constantes producen valores bajos.

1.1 ACI temporal (ACItf)

Se calcula la suma de las diferencias absolutas entre valores consecutivos en el tiempo, normalizadas por la energía:

ACItf = Σ ( |S(f, t+1) – S(f, t)| / (S(f, t+1) + S(f, t) + ε) )

1.2 ACI frecuencial (ACIft)

Se calcula la variación a lo largo de la frecuencia, por cada instante de tiempo:

ACIft = Σ ( |S(f+1, t) – S(f, t)| / (S(f+1, t) + S(f, t) + ε) )

Notas en ultrasonido:
Matemáticamente aplicable, pero con cautela: los pulsos muy breves (como los de murciélagos) pueden inflar el valor. Conviene ajustar tamaño de ventana, NFFT y solapamiento, y documentar siempre la banda analizada.

2) BI — Bioacoustics Index

Intuición.
Resume la energía promedio dentro de una banda de frecuencias definida como “biológica”.

Procedimiento:

Calcular el espectro medio promediando el espectrograma en el tiempo.

Sumar los valores de ese espectro dentro de la banda biológica (por ejemplo, 2–12 kHz).

Notas en ultrasonido:
En su definición original no aplica. Puede redefinirse para bandas ultrasónicas (por ejemplo, 20–80 kHz), pero pierde comparabilidad con la literatura audible. Debe documentarse explícitamente si se redefine.

3) NDSI — Normalized Difference Soundscape Index

Intuición.
Compara la energía en dos bandas: biophony (bio) y anthrophony (antro).

NDSI = (Pbio – Pantro) / (Pbio + Pantro)

Donde Pbio es la energía en la banda biológica, y Pantro la energía en la banda antropogénica.

Notas en ultrasonido:
No existen bandas “antro” estándar en ultrasonido, por lo cual no es recomendable en su forma clásica.

4) NP — Number of Peaks (Número de picos)

Intuición.
Cuenta cuántos picos locales existen en el espectro (o espectrograma colapsado en frecuencia) dentro de la banda analizada.

Procedimiento:

Colapsar el espectrograma en frecuencia (sumar sobre el tiempo).

Detectar máximos locales aplicando un criterio de distancia mínima en frecuencia y umbral de prominencia.

NP = número de picos detectados.

Notas en ultrasonido:
Es válido y útil, pero hay que ajustar los parámetros de distancia mínima en frecuencia y los umbrales según la resolución y las especies objetivo.

5) Entropías acústicas (Ht, Hf y H)

Intuición.
Miden la dispersión o heterogeneidad temporal y espectral de la energía.

5.1 Entropía temporal (Ht)

Calcular la envolvente temporal (por ejemplo, con transformada de Hilbert).

Normalizar la energía en el tiempo como una distribución de probabilidad.

Calcular la entropía de Shannon normalizada (0 = toda la energía concentrada en un instante; 1 = energía distribuida uniformemente).

5.2 Entropía frecuencial (Hf)

Colapsar el espectrograma en frecuencia (sumar sobre el tiempo).

Normalizar como distribución de probabilidad en frecuencia.

Calcular la entropía de Shannon normalizada.

5.3 Entropía total (H)

Se define como el producto: H = Ht * Hf

Notas en ultrasonido:
Válidas mientras se definan bien los parámetros del STFT y la banda de análisis. Son métricas agnósticas de la frecuencia.

# INSTALACIÓN PAQUETES

In [ ]:
# ===========================
# 0) Instalación de paquetes
# ===========================
!pip -q install numpy==1.26.4
!pip -q install scikit-maad soundfile tqdm pandas scipy openpyxl
!apt -y -qq install ffmpeg > /dev/null


# Soporte para formatos comprimidos con torchaudio (mp3/m4a/ogg a través de ffmpeg)
!apt -y -qq install ffmpeg > /dev/null



# MONTAR DRIVE

In [ ]:
# ===========================
# 1) Montar Google Drive
# ===========================
from google.colab import drive
drive.mount('/content/drive')


# CONFIGURAR PARÁMETROS

Definir parámetros para índices acústicos
Window_Size (Tamaño de ventana): El tamaño de ventana (como WINDOW_SIZE = 512) determina cuántos puntos de la señal se analizan en cada segmento al calcular un espectrograma con la Transformada Rápida de Fourier (FFT). Es clave porque afecta la resolución temporal y espectral del análisis.

Tamaños pequeños (e.g., 256 puntos): Ofrecen mejor resolución temporal, lo que permite capturar eventos rápidos en el tiempo, pero sacrifican precisión en la frecuencia (bandas más amplias). Tamaños grandes (e.g., 1024 puntos): Mejoran la resolución espectral, lo que permite identificar frecuencias específicas con mayor precisión, pero sacrifican detalles temporales, dificultando la captura de eventos breves. Un tamaño de 512 puntos es común, equilibrando la capacidad de capturar eventos en tiempo y frecuencia. En combinación con el solapamiento, permite mantener continuidad entre ventanas consecutivas y obtener un espectrograma más detallado y suave.

Overlap (sobrelapamiento): El solapamiento (overlap) es la cantidad de datos compartidos entre ventanas consecutivas de una señal al calcular su espectrograma mediante la Transformada Rápida de Fourier (FFT). Esto implica que, al dividir una señal en ventanas de un tamaño específico (por ejemplo, 512 puntos), una nueva ventana comienza antes de que termine la anterior, permitiendo que se superpongan. Esto ayuda a capturar transiciones rápidas y mejora la continuidad temporal en el análisis. Por ejemplo, con un solapamiento del 50% y una ventana de 1024 puntos, la segunda ventana comenzará en el punto 512 de la primera y continuará superponiéndose.

El solapamiento es útil porque mejora la resolución temporal, reduce la pérdida de información entre ventanas y genera espectrogramas más suaves visualmente.

Solapamiento bajo (< 50%): Puede ser útil si quieres reducir el tiempo de cómputo o si los eventos son lentos. Solapamiento alto (> 75%): Ideal para señales con cambios rápidos en el tiempo, pero aumenta el costo computacional. Solapamiento medio (50%): Es una configuración estándar que equilibra bien entre resolución temporal y computación (David Luna, com.pers).

NFFT (Puntos FFT): El número de puntos FFT determina cuántos puntos se calculan en la Transformada Rápida de Fourier (FFT) para cada ventana. Esto afecta directamente la resolución en el dominio de la frecuencia:

Valores pequeños (e.g., 256): Proporcionan menos precisión en las frecuencias (bandas más amplias), pero son más rápidos de calcular. Valores grandes (e.g., 1024): Ofrecen mayor resolución en frecuencia, dividiendo el espectro en bandas más estrechas, pero requieren más cómputo. Un valor de NFFT = 512 es común porque equilibra la precisión en la frecuencia con la eficiencia computacional. Normalmente se elige igual al tamaño de la ventana (WINDOW_SIZE) para mantener coherencia en el análisis.

J_CLUSTER (El número de clusters para el ACI temporal) Define cómo se divide la señal en bloques de tiempo al calcular el Índice de Complejidad Acústica (ACI) temporal.

Cada cluster representa un segmento de tiempo sobre el cual se evalúan los cambios en la energía de las frecuencias. Este valor afecta la granularidad del análisis temporal:

Valores bajos (ej., 2-5): Agrupan más datos en cada cluster, lo que suaviza el análisis temporal y detecta tendencias generales.

Valores altos (ej., >10): Aumentan la sensibilidad a cambios rápidos en la señal, capturando eventos más pequeños o transitorios.

In [11]:
import os, warnings
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from scipy import signal
from tqdm import tqdm
import concurrent.futures

warnings.filterwarnings("ignore")

# Carpeta AUDIOS en tu Drive (ajusta si usas 'My Drive' en lugar de 'MyDrive')
BASE_PATH = Path('/content/drive/MyDrive/AUDIOS')
if not BASE_PATH.exists():
    BASE_PATH = Path('/content/drive/My Drive/AUDIOS')
assert BASE_PATH.exists(), f"No encuentro la carpeta: {BASE_PATH}"

# Archivo de salida (en la misma carpeta AUDIOS)
OUTPUT_FILE = BASE_PATH / f"IndicesAcusticos_0-24_{datetime.now().strftime('%Y%m%d')}.xlsx"

# Espectrograma
WINDOW_SIZE = 512
NFFT        = 512
OVERLAP     = 0

# Bandas/índices
DEFAULT_FMIN     = 0
DEFAULT_FMAX     = 24000
BIOPHONY_BAND    = (2000, 12000)
ANTHROPHONY_BAND = (0, 2000)

# Clustering temporal opcional (promedio de J ventanas contiguas ANTES de ACI temporal)
J_CLUSTER = 5  # usar 1 para no agrupar;  p.ej. 5

# Paralelismo
WORKERS = max(1, os.cpu_count() - 1)
USE_PROCESSES = False  # En notebooks, los hilos son más estables que procesos
AUDIO_EXTS = {'.wav'}  # respeta tu patrón original centrado en WAV


# Calcular índices

In [17]:
import soundfile as sf
from maad import features

EPS = 1e-10

def load_audio_sf(file_path: Path):
    """
    Carga audio con soundfile -> (waveform, samplerate)
    waveform: ndarray [N] (solo primer canal si es estéreo).
    """
    data, sr = sf.read(str(file_path), dtype='float32')
    if data.ndim > 1:
        data = data[:, 0]  # tomar primer canal
    return data, sr

def ACItf_from_Sxx(Sxx, f, fmin, fmax, J=1):
    mask = (f >= fmin) & (f <= fmax)
    S = Sxx[mask, :]
    if S.size == 0 or S.shape[1] < 2: return np.nan
    if J > 1:
        T = S.shape[1]
        n = (T // J) * J
        if n > 0:
            S = S[:, :n].reshape(S.shape[0], n//J, J).mean(axis=2)
    num = np.abs(np.diff(S, axis=1)).sum(axis=1)
    den = (S.sum(axis=1) + EPS)
    return float(np.nansum(num / den))

def ACIft_from_Sxx(Sxx, f, fmin, fmax):
    mask = (f >= fmin) & (f <= fmax)
    S = Sxx[mask, :]
    if S.size == 0 or S.shape[0] < 2: return np.nan
    num = np.abs(np.diff(S, axis=0)).sum(axis=0)
    den = (S.sum(axis=0) + EPS)
    return float(np.nansum(num / den))

def number_of_peaks(Sxx, f, fmin, fmax, min_freq_dist=200):
    mask = (f >= fmin) & (f <= fmax)
    S = Sxx[mask, :]
    if S.size == 0: return np.nan
    try:
        return float(features.number_of_peaks(S, f[mask], flim=(fmin,fmax), min_freq_dist=min_freq_dist))
    except Exception:
        return np.nan

def calculate_H(x, Fs, s, f, fmin, fmax):
    # Ht
    hilbert_env = np.abs(signal.hilbert(x))
    env_sum = hilbert_env.sum()
    if env_sum <= 0: Ht = 0.0
    else:
        env_prob = hilbert_env / env_sum
        Ht = -np.sum(env_prob * np.log2(env_prob+EPS)) / np.log2(len(env_prob))
    # Hf
    mask = (f >= fmin) & (f <= fmax)
    s_band = s[mask, :].sum(axis=1)
    den = np.sum(s_band)
    if den <= 0: Hf = 0.0
    else:
        s_band = s_band / (den+EPS)
        Hf = -np.sum(s_band*np.log2(s_band+EPS)) / np.log2(len(s_band))
    return float(Ht), float(Hf), float(Ht*Hf)

def get_indices(file_path, fmin, fmax):
    try:
        x, fs = load_audio_sf(file_path)
        f, t, s = signal.spectrogram(x, fs, nperseg=WINDOW_SIZE, noverlap=OVERLAP, nfft=NFFT, mode='magnitude')
        ACItf_val = ACItf_from_Sxx(s, f, fmin, fmax, J=J_CLUSTER)
        ACIft_val = ACIft_from_Sxx(s, f, fmin, fmax)
        BI_val    = float(features.bioacoustics_index(s, f, flim=BIOPHONY_BAND))
        NP_val    = number_of_peaks(s, f, fmin, fmax)
        Ht_val,Hf_val,H_val = calculate_H(x, fs, s, f, fmin, fmax)
        NDSI_val  = float(features.soundscape_index(s, f, flim_bioPh=BIOPHONY_BAND,
                                                   flim_antroPh=ANTHROPHONY_BAND)[0])
        return [file_path.name, file_path.parent.name, ACItf_val, ACIft_val, BI_val, NP_val, Ht_val, Hf_val, H_val, NDSI_val]
    except Exception as e:
        print(f"❌ Error procesando {file_path}: {e}")
        return [file_path.name, file_path.parent.name] + [np.nan]*8


# CORRER EXPORTAR RESULTADOS

In [ ]:
files = [p for p in BASE_PATH.rglob('*') if p.is_file() and p.suffix.lower()=='.wav']
print(f"Número de WAV: {len(files)}")
if not files: raise SystemExit("No hay archivos WAV en AUDIOS")

with concurrent.futures.ThreadPoolExecutor(max_workers=WORKERS) as ex:
    results = list(tqdm(ex.map(get_indices, files, [DEFAULT_FMIN]*len(files), [DEFAULT_FMAX]*len(files)),
                        total=len(files), desc="Procesando archivos"))

df = pd.DataFrame(results, columns=["Name","Folder","ACItf","ACIft","BI","NP","Ht","Hf","H","NDSI"])
df.to_excel(OUTPUT_FILE, index=False)
print("Resultados guardados en:", OUTPUT_FILE)
